<a href="https://colab.research.google.com/github/sightes/Scraping/blob/main/https_clinicasdelazucar_com_contacto_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El presente codigo realiza un scrapping completo de los productos cargados en diartek.cl, tener cuidado con los precios dado que existen algunos q se encuentran en dolares sin embargo en el sitio web no existe referencia a la moneda

In [1]:
%%capture
!pip install selenium
!pip install fake_useragent

In [2]:
import pandas as pd
import json
from selenium.webdriver.common.by import By
from selenium import webdriver
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import numpy as np


def try_or(func, default=None, expected_exc=(Exception,)):
    try:
        return func()
    except expected_exc:
        return default

def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    #options.add_argument("--incognito")
    options.add_argument("--nogpu")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1280,1280")
    options.add_argument("--no-sandbox")
    options.add_argument("--user-data-dir=chrome-data")
    options.add_argument("--enable-javascript")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
    ua = UserAgent()
    userAgent = ua.random
    driver = webdriver.Chrome(options=options)
    driver.execute_cdp_cmd('Network.setUserAgentOverride',{"userAgent": userAgent,'method': 'GET','Host':'www.solili.mx'})
    driver.set_network_conditions(offline=False,
                                  latency=5,
                                  download_throughput=500 * 1024,
                                  upload_throughput=500 * 1024)
    return(driver)
def get_atributes(a):
  return([float(a.get_attribute('data-price').replace('.','').replace(',','.').replace('$','')),
  float(a.get_attribute('data-price-default').replace('.','').replace(',','.').replace('$','')),
  a.get_attribute('data-brand'),
  a.get_attribute('data-name'),
  'https://diartek.cl/'+a.get_attribute('data-permalink'),
  a.get_attribute('data-image'),
  a.get_attribute('data-id')])


In [36]:
driver=create_driver()
driver.get('https://clinicasdelazucar.com/contacto/')
ubicaciones=driver.find_elements(By.XPATH,"//*[contains(@class,'col-lg-4 col-md-6 col-xs-12')]")

In [32]:
def get_redirected(url):
  import requests
  response = requests.get(url, headers={'User-Agent': 'Google Chrome'})
  row = {}
  if response.history:
      for step in response.history:
          row['original_url'] = step.url
          row['original_status'] = step.status_code
      row['destination_url'] = response.url
      row['destination_status'] = response.status_code
  else:
      row['original_url'] = response.url
      row['original_status'] = response.status_code
      row['destination_url'] = ''
      row['destination_status'] = ''
  return(row)

In [37]:
data=[]
for i in range(0,len(ubicaciones)):
  name=ubicaciones[i].find_elements(By.XPATH,".//*[contains(@class,'card-header  text-lg-center')]/b")[0].text
  direccion=ubicaciones[i].find_elements(By.XPATH,".//*[contains(@class,'card-body')]/p")[0].text
  telefono=ubicaciones[i].find_elements(By.XPATH,".//*[contains(@class,'card-body')]/a")[0].text
  url=ubicaciones[i].find_elements(By.XPATH,".//*[contains(@align,'center')]/a")[0].get_attribute('href')
  latitude=get_redirected(url)['destination_url'].split('@')[1].split(",")[0]
  longitude=get_redirected(url)['destination_url'].split('@')[1].split(",")[1]
  data=data+[[name,direccion,telefono,url,latitude,longitude]]


In [40]:
pd.DataFrame(data).to_csv('data.csv')